# Análises estatísticas para gestão de processos

Este notebook foi desenvolvido por [Paulo Araujo](mailto:paulo@visiumlabs.com.br) e tem por objetivo demonstrar a diferença de se trabalhar com uma função de distribuição para modelamento de um sistema a fim de se realizar análises estratégicas do mesmo.

## Pacotes do Python

Neste exemplo utilizaremos a biblioteca [Numpy](https://numpy.org) para geração de números aleatórios e cálculos estatísticos.

In [5]:
import numpy as np

Definindo algumas variáveis de controle do processo.

In [6]:
HORAS = 24 * 30
OBSERVACOES = [10,100,1000]
MEDIA = 0.1
DESVP = 0.02
LIM_INF = 0.02
LIM_SUP = 0.18
SEED = 0

Como estamos trabalhando com uma função de distribuição, nosso modelo passa a ser um modelo estocástico. Portanto, para avaliarmos o processo é necessário simular aleatoriamente várias vezes um mesmo cenário. Nesse exemplo, poderemos verificar as diferenças estatísticas obtidas rodando o cenário 10, 100 e 1000 vezes.

In [7]:
# Gerando amostras e calculando a produtividade
# em termos dos valores médios de cada amostra
P = np.zeros((2,len(OBSERVACOES)))
for i,obs in enumerate(OBSERVACOES):
    # Array para guardar números observados
    N = np.zeros((obs,1))
    for j in range(obs):
        # Ajustando a semente do gerador de números aleatórios
        SEED += 1
        np.random.seed(SEED)
        # Gerando muitos números aleatórios
        s = np.random.normal(MEDIA,DESVP,(int(HORAS//MEDIA * 1.5),))
        # Procedimento para garantir que apenas tempos
        # dentro de um intervalo pré-definido
        c = np.logical_and(s >= LIM_INF, s <= LIM_SUP)
        while not np.all(c):
            d = c.shape[0] - np.count_nonzero(c)
            s[c == False] = np.random.normal(MEDIA,DESVP,(d,))
            c = np.logical_and(s >= LIM_INF, s <= LIM_SUP)
        # Calculando a produtividade
        prod = (np.cumsum(s)<=HORAS).argmin()
        # Armazenando estatísticas do experimento
        N[j,:] = prod
    # Armazenando estatísticas finais de todas as observações
    P[:,i] = np.array([np.mean(N),np.std(N)])

Imprimindo os resultados.

In [8]:
for i in range(P.shape[1]):
    print(f'Número de observações: {OBSERVACOES[i]} => Produtividade Média: {P[0,i]} => Desvio-padrão: {P[1,i]}')

Número de observações: 10 => Produtividade Média: 7205.8 => Desvio-padrão: 22.85957129956728
Número de observações: 100 => Produtividade Média: 7197.4 => Desvio-padrão: 15.974354447050436
Número de observações: 1000 => Produtividade Média: 7199.139 => Desvio-padrão: 16.99210637325461
